In [0]:
#rm -rf oncemilcuatroclases.zip

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.utils import to_categorical
import pickle
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import glob
from keras.callbacks import ModelCheckpoint
from keras import models
from keras.models import load_model



Using TensorFlow backend.


In [0]:
!unzip /content/oncemilcuatroclases.zip

Archive:  /content/oncemilcuatroclases.zip
   creating: cwm/
  inflating: cwm/1.jpg               
  inflating: cwm/2.jpg               
  inflating: cwm/3.jpg               
  inflating: cwm/8.jpg               
  inflating: cwm/9-3.jpg             
  inflating: cwm/9-4.jpg             
  inflating: cwm/9-9.jpg             
  inflating: cwm/9-10.jpg            
  inflating: cwm/9-12.jpg            
  inflating: cwm/9-13.jpg            
  inflating: cwm/9-14.jpg            
  inflating: cwm/9-18.jpg            
  inflating: cwm/9-20.jpg            
  inflating: cwm/9-21.jpg            
  inflating: cwm/9-22.jpg            
  inflating: cwm/9-23.jpg            
  inflating: cwm/9-25.jpg            
  inflating: cwm/9-26.jpg            
  inflating: cwm/9-27.jpg            
  inflating: cwm/9-30.jpg            
  inflating: cwm/9-31.jpg            
  inflating: cwm/9-33.jpg            
  inflating: cwm/9-34.jpg            
  inflating: cwm/9-38.jpg            
  inflating: cwm/9-40.jpg  

In [0]:
#create list of file paths
paths = []
for filepath in glob.glob('/content/*/*'):
    paths.append(filepath)

In [0]:
def loadImageAsGray(imagePath):
    im = cv2.imread(imagePath)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    return im

In [0]:
def getData(path):
    
    dic = {"array":[],"tag":[]}
    for foto in glob.iglob(path):
        foto_new = loadImageAsGray(foto)
        foto = foto.split("/")
        foto_array = cv2.imread(filepath, 0)
        dic['tag'].append(foto[-2])
        dic['array'].append(foto_new)
                
    df = pd.DataFrame(dic)
    new = pd.get_dummies(df['tag'])
    df_gt = df.join(new)
    return df_gt

In [0]:
data = getData('/content/*/*.jpg')

In [0]:
data

,array,tag,cwm,jwm,kwm,otherwm2
0,"[[220, 212, 189, 167, 149, 126, 113, 113, 120,...",cwm,1,0,0,0
1,"[[85, 85, 86, 87, 87, 87, 86, 85, 86, 86, 86, ...",cwm,1,0,0,0
2,"[[125, 124, 123, 121, 119, 120, 120, 117, 112,...",cwm,1,0,0,0
3,"[[75, 75, 75, 70, 64, 65, 68, 65, 62, 62, 62, ...",cwm,1,0,0,0
4,"[[29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, ...",cwm,1,0,0,0
...,...,...,...,...,...,...
11106,"[[108, 111, 115, 121, 128, 138, 149, 158, 154,...",jwm,0,1,0,0
11107,"[[10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, ...",jwm,0,1,0,0
11108,"[[113, 112, 110, 106, 102, 97, 92, 89, 86, 81,...",jwm,0,1,0,0
11109,"[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",jwm,0,1,0,0


In [0]:
#data1 = data.drop(['tag'], axis = 1)
data['gt'] = data[['cwm', 'jwm', 'kwm', 'otherwm2']].apply(lambda x: pd.Series([x.values]), axis=1)

In [0]:
data

,array,tag,cwm,jwm,kwm,otherwm2,gt
0,"[[220, 212, 189, 167, 149, 126, 113, 113, 120,...",cwm,1,0,0,0,"[1, 0, 0, 0]"
1,"[[85, 85, 86, 87, 87, 87, 86, 85, 86, 86, 86, ...",cwm,1,0,0,0,"[1, 0, 0, 0]"
2,"[[125, 124, 123, 121, 119, 120, 120, 117, 112,...",cwm,1,0,0,0,"[1, 0, 0, 0]"
3,"[[75, 75, 75, 70, 64, 65, 68, 65, 62, 62, 62, ...",cwm,1,0,0,0,"[1, 0, 0, 0]"
4,"[[29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, ...",cwm,1,0,0,0,"[1, 0, 0, 0]"
...,...,...,...,...,...,...,...
11106,"[[108, 111, 115, 121, 128, 138, 149, 158, 154,...",jwm,0,1,0,0,"[0, 1, 0, 0]"
11107,"[[10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, ...",jwm,0,1,0,0,"[0, 1, 0, 0]"
11108,"[[113, 112, 110, 106, 102, 97, 92, 89, 86, 81,...",jwm,0,1,0,0,"[0, 1, 0, 0]"
11109,"[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",jwm,0,1,0,0,"[0, 1, 0, 0]"


In [0]:
num_classes = 4
# input image dimensions
img_rows, img_cols, channels = 200, 200, 1
X = np.stack(data['array'])
y = np.stack(data['gt'])
# the data, split between train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
input_shape = (img_rows, img_cols, channels)
# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
# convert class vectors to binary class matrices
#y_train =to_categorical(y_train)

(8888, 200, 200)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [0]:
print(X_train.shape)

(8888, 200, 200)


In [0]:
input_shape = (img_rows, img_cols, 1)

In [0]:
y_train[:5]

array([[0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1]], dtype=uint8)

In [0]:
# Prepare data to feed the NN
num_classes = 4

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (8888, 200, 200, 1)
8888 train samples
2223 test samples


In [0]:
## This is the neural network proposed architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
filepath='Checkpoint_{epoch:02d}_{val_acc:.2f}'
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
# Fit the NN
batch_size = 20
epochs = 10

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 8888 samples, validate on 2223 samples
Epoch 1/10





8888/8888 [==============================] - 1866s 210ms/step - loss: 1.9305 - acc: 0.5287 - val_loss: 0.9369 - val_acc: 0.5857

Epoch 00001: val_acc improved from -inf to 0.58570, saving model to Checkpoint_01_0.59
Epoch 2/10
8888/8888 [==============================] - 1857s 209ms/step - loss: 0.8413 - acc: 0.6519 - val_loss: 0.7645 - val_acc: 0.6968

Epoch 00002: val_acc improved from 0.58570 to 0.69681, saving model to Checkpoint_02_0.70
Epoch 3/10
8888/8888 [==============================] - 1852s 208ms/step - loss: 0.6316 - acc: 0.7412 - val_loss: 0.5888 - val_acc: 0.7796

Epoch 00003: val_acc improved from 0.69681 to 0.77958, saving model to Checkpoint_03_0.78
Epoch 4/10
8888/8888 [==============================] - 1858s 209ms/step - loss: 0.4792 - acc: 0.8113 - val_loss: 0.4951 - val_acc: 0.8259

Epoch 00004: val_acc im

In [0]:
import time

moment=time.localtime()
name='Model_{}_{}-{}-{}.h5'.format(model.history.history["val_acc"][-1],moment[2],moment[3],moment[4])
model.save(name)

In [0]:
import json

model_json = model.to_json()
with open(name+'.json', "w") as json_file:
    json.dump(model_json, json_file)

model.save_weights(name+'.h5')

In [0]:
new_model = load_model('/content/drive/My Drive/Colab Notebooks/chinojapocoreano/modelos guardados/Model_0.8317588851680318_18-13-3.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
batch_size = 20
epochs = 2

new_model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_list)

Train on 8888 samples, validate on 2223 samples
Epoch 1/2
6040/8888 [===================>..........] - ETA: 9:42 - loss: 0.3651 - acc: 0.8623

In [0]:
import time
import json

moment=time.localtime()
name='Model_{}_{}-{}-{}.h5'.format(new_model.history.history["val_acc"][-1],moment[2],moment[3],moment[4])
new_model.save(name)

import json

new_model_json = new_model.to_json()
with open(name+'.json', "w") as json_file:
    json.dump(new_model_json, json_file)

new_model.save_weights(name+'.h5')

NameError: ignored

In [0]:
new_model1 = load_model('/content/Model_0.9239766018593359_18-17-40.h5')

In [0]:
batch_size = 20
epochs = 2

new_model1.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),
          callbacks=callbacks_list)

Train on 8888 samples, validate on 2223 samples
Epoch 1/2
 840/8888 [=>............................] - ETA: 28:08 - loss: 0.1021 - acc: 0.9655

KeyboardInterrupt: ignored